In [4]:
!pip install --no-cache-dir -q opencv-python requests tensorflow_hub matplotlib

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import cv2
import requests
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import shutil

%matplotlib inline

print("Tensorflow Version: {}".format(tf.__version__))
print("Tensorflow.Keras Version: {}".format(keras.__version__))

Tensorflow Version: 1.15.2
Tensorflow.Keras Version: 2.2.4-tf


# Helpers

In [6]:
def preprocess_image(img_path):
  img = cv2.imread(img_path)
  img = img[:,:,::-1]
  img = cv2.resize(img, (224,224))
  plt.imshow(img)
  plt.show()
  _img = img / 255.0
  return _img, img

In [ ]:
#cls_img_path = "/Users/jiankaiwang/devops/VOCdevkit/VOC2008/JPEGImages/2007_000648.jpg"
cls_img_path = "/Users/jiankaiwang/devops/VOCdevkit/VOC2008/JPEGImages/2007_000033.jpg"
assert os.path.exists(cls_img_path), "Image was not found."
img, img_ori = preprocess_image(cls_img_path)
img = np.expand_dims(img, axis=0)
img_ori = np.expand_dims(img_ori, axis=0)
assert img.shape == (1,224,224,3), "Image shape was not allowed."

In [ ]:
label_raw_data = requests.get("https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt")
labels = []
_tmp_word = ""
for word in label_raw_data.content.decode('utf-8'):
  if word not in ["\n", "\r\n"]:
    _tmp_word += word
  else:
    labels.append(_tmp_word)    
    _tmp_word = ""
print(len(labels), labels[0], labels[-1])

# SavedModel

In [ ]:
saved_model_path = "/Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification"
assert os.path.exists(saved_model_path), "Saved model path was not found."

## Using tf.saved_model APIs (SavedModel)

### Customized Model

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/MNIST_data", one_hot=True)

In [56]:
full_training = True
learning_rate = 1e-2
training_epochs = 2 if full_training else 1
batch_size = 64 if full_training else 32
display_step = 1
log_path = os.path.join("/","tmp","tensorboard_log","mnist_example")

In [5]:
def conv2d(input, weight_shape, bias_shape):
  in_count = weight_shape[0] * weight_shape[1] * weight_shape[2]
  weight_init = tf.random_normal_initializer(stddev=(2/in_count)**0.5)
  bias_init = tf.constant_initializer(value=0)
  weight = tf.get_variable("W", weight_shape, initializer=weight_init)
  bias = tf.get_variable("b", bias_shape, initializer=bias_init)
  conv = tf.nn.conv2d(input, weight, strides=[1, 1, 1, 1], padding="SAME")
  return tf.nn.relu(tf.nn.bias_add(conv, bias))

In [6]:
def pool2d(input, k=2):
  return tf.nn.max_pool(value=input, ksize=[1,k,k,1], strides=[1,k,k,1], padding="SAME")

In [7]:
def dense(input, weight_shape, bias_shape, name):
  in_count = weight_shape[0] * weight_shape[1]
  weight_init = tf.random_normal_initializer(stddev=(2/in_count)**0.5)
  bias_init = tf.constant_initializer(value=0)
  weight = tf.get_variable("W", weight_shape, initializer=weight_init)
  bias = tf.get_variable("b", bias_shape, initializer=bias_init)
  logits = tf.matmul(input, weight)
  return tf.nn.relu(tf.nn.bias_add(logits, bias), name=name)

In [8]:
def inference(input, keep_prob=0.5):
  x = tf.reshape(input, [-1, 28, 28, 1])

  with tf.variable_scope("hidden_1"):
    conv_1 = conv2d(x, [3, 3, 1, 32], [32])         # 28 x 28 x 32
    pool_1 = pool2d(conv_1, 2)                      # 14 x 14 x 32

  with tf.variable_scope("hidden_2"):
    conv_2 = conv2d(pool_1, [3, 3, 32, 64], [64])   # 14 x 14 x 64
    pool_2 = pool2d(conv_2, 2)                      # 7 x 7 x 64

  with tf.variable_scope("fc"):
    pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
    fc_1 = dense(pool_2_flat, [7 * 7 * 64, 1024], [1024], name="fc")

    # dropout
    fc_1_dropout = tf.nn.dropout(fc_1, keep_prob=keep_prob)

  with tf.variable_scope("output"):
    output = dense(fc_1_dropout, [1024, 10], [10], name="logits")

  return output

In [9]:
def loss(output, y):
  """
  output: the logits value from inference
  y: the labeling data
  """
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
  loss = tf.reduce_mean(cross_entropy)
  return loss

In [10]:
def training(loss, global_step):
  """
  loss: the loss value
  global_step: the global training step index
  """
  tf.summary.scalar("loss", loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  grads = tf.gradients(loss, tf.trainable_variables())
  grads = list(zip(grads, tf.trainable_variables()))
  apply_grads = optimizer.apply_gradients(grads_and_vars=grads, global_step=global_step)
  return grads, apply_grads

In [11]:
def evaluate(output, y):
  """
  output: the logits value from inference
  y: the labeling data
  """
  compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
  accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
  tf.summary.scalar("eval", accuracy)
  return accuracy

In [55]:
!rm -rf {saved_model_path}

In [58]:
saved_model_path = "/Users/jiankaiwang/devops/tmp/tf_saved_model_apis"
if os.path.exists(saved_model_path):
  shutil.rmtree(saved_model_path)

tf.reset_default_graph()

with tf.Graph().as_default():
  with tf.variable_scope("cnn"):
    x = tf.placeholder("float", [None, 784], name="images")
    y = tf.placeholder("float", [None, 10], name="label")
    keep_prob = tf.placeholder(tf.float32)

    output = inference(x, keep_prob=keep_prob) 
    loss_val = loss(output=output, y=y)
    global_step = tf.Variable(0, name="global_step", trainable=False)
    train_grads, train_opt = training(loss=loss_val, global_step=global_step)   # training body
    eval_opt = evaluate(output=output, y=y)   # evaluation result

    with tf.Session() as sess:

      sess.run(tf.global_variables_initializer())   # initialize all variables

      for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(mnist.train.num_examples / batch_size)

        for idx in range(total_batch):
          batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)   # get the batch data

          feed_dict_data = {x: batch_x, y: batch_y, keep_prob: 0.5}
          grads, _ = sess.run([train_grads, train_opt], feed_dict=feed_dict_data)   # run training

          batch_loss = sess.run(loss_val, feed_dict=feed_dict_data)
          avg_loss += batch_loss / total_batch   # calculate the average loss

        if epoch % display_step == 0:
          feed_dict_val_data = {x: mnist.validation.images, y: mnist.validation.labels, keep_prob: 1.0}
          acc = sess.run(eval_opt, feed_dict=feed_dict_val_data)   # calculate the accuracy
          print("Epoch: {}, Accuracy: {:.2f}, Vaildation Error: {:.2f}".format(epoch+1, acc, 1-acc))

      print("Training finishing.")

      feed_dict_test_data = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}
      acc = sess.run(eval_opt, feed_dict=feed_dict_test_data)
      print("Test Accuracy: {:.2f}".format(acc))

      tf.saved_model.simple_save(sess, saved_model_path, inputs={"image":x, "rate": keep_prob}, outputs={"output": output})

Epoch: 1, Accuracy: 0.93, Vaildation Error: 0.07
Epoch: 2, Accuracy: 0.95, Vaildation Error: 0.05
Training finishing.
Test Accuracy: 0.95
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /Users/jiankaiwang/devops/tmp/tf_saved_model_apis/saved_model.pb


## Load the Model

In [67]:
saved_model_path = "/Users/jiankaiwang/devops/tmp/tf_saved_model_apis"
assert os.path.exists(saved_model_path), "The model folder was not found."

tf.Graph()

with tf.Session(graph=graph) as sess:
  tf.saved_model.load(sess, ['serve'], saved_model_path)

  # show all operations
  all_ops = graph.get_operations()
  for ops in all_ops[:10]:
    if ops.name[:8] != "cnn/save": print(ops.name)

  inputs = graph.get_tensor_by_name("cnn/images:0")
  outputs = graph.get_tensor_by_name("cnn/output/logits:0")
  keep_prob = graph.get_tensor_by_name("cnn/Placeholder:0")

  val_idx = 3
  inference_result = sess.run(outputs, feed_dict={
    inputs: mnist.test.images[val_idx:(val_idx+1)],
    keep_prob: 1.0})
  print("Label: {}".format(np.argmax(mnist.test.labels[val_idx])))
  print("Inference result: {}".format(np.argmax(inference_result[0])))
  print(inference_result)

INFO:tensorflow:Restoring parameters from /Users/jiankaiwang/devops/tmp/tf_saved_model_apis/variables/variables
cnn/images
cnn/label
cnn/Placeholder
cnn/Reshape/shape
cnn/Reshape
cnn/hidden_1/W/Initializer/random_normal/shape
cnn/hidden_1/W/Initializer/random_normal/mean
cnn/hidden_1/W/Initializer/random_normal/stddev
cnn/hidden_1/W/Initializer/random_normal/RandomStandardNormal
cnn/hidden_1/W/Initializer/random_normal/mul
Label: 0
Inference result: 0
[[12.604624   0.         3.8232436  0.6547819  0.         3.2755642
   3.911615   1.3226465  1.1137877  2.6310313]]


## Using Models from tf.hub

In [ ]:
saved_model_path = "/Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification"
assert os.path.exists(saved_model_path), "Saved model path was not found."

In [ ]:
!saved_model_cli show --dir /Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification

In [ ]:
from tensorflow_hub import registry

In [ ]:
def resolve(handle):
  """Resolves a module handle into a path.
   Resolves a module handle into a path by downloading and caching in
   location specified by TF_HUB_CACHE_DIR if needed.
  Args:
    handle: (string) the Module handle to resolve.
  Returns:
    A string representing the Module path.
  """
  return registry.resolver(handle)

In [ ]:
saved_model_path = "/Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification"
assert os.path.exists(saved_model_path), "Saved model path was not found."

tf.reset_default_graph()

with tf.Graph().as_default() as graph:
  module_handle = resolve(saved_model_path)
  loaded = tf.saved_model.load_v2(module_handle, tags=['train'])   
  all_ops = graph.get_operations()
  f = loaded.signatures["image_classification"]

  with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    res = sess.run(f(images=tf.convert_to_tensor(img, dtype=tf.float32)))

In [ ]:
saved_model_path = "/Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification"
assert os.path.exists(saved_model_path), "Saved model path was not found."

tf.reset_default_graph()

with tf.Graph().as_default() as graph:
  with tf.Session(graph=graph) as sess:
    loaded = tf.saved_model.load(sess, [], saved_model_path)   

    all_ops = graph.get_operations()
    #for ops in all_ops:
    #    print(ops.name) 

    inputs = graph.get_tensor_by_name("hub_input/images:0")
    outputs = graph.get_tensor_by_name("MobilenetV2/Logits/output:0")

    res = sess.run(outputs, feed_dict={inputs: img})
    res_idx = np.argmax(res[0])
    print("Result : {}, Prob: {}".format(labels[res_idx], res[0][res_idx]))

# Using tf.hub APIs (SavedModel)

In [ ]:
saved_model_path = "/Users/jiankaiwang/devops/tfhub/imagene_mobilenet_v2_140_224_classification"
assert os.path.exists(saved_model_path), "Saved model path was not found."

In [ ]:
with tf.Graph().as_default():  
  classifier = hub.Module(saved_model_path)
  height, width = hub.get_expected_image_size(classifier)
  #tf.logging.info("Suggested height {}, width {}".format(height, width))
  
  images = tf.placeholder(tf.float32, shape=(1, height, width, 3))  
  logits = classifier(images)

  with tf.Session() as sess:
    collects = tf.group(tf.global_variables_initializer(), 
                        tf.tables_initializer())
    sess.run(collects)
    
    cls_res, = sess.run([logits], feed_dict={images: img})
    cls_res = cls_res[0]
    cls_res_idx = np.argmax(cls_res)
    print("Classification result: {}, prob: {}".format(labels[cls_res_idx], cls_res[cls_res_idx]))

# Frozen Model

In [ ]:
frozen_model_path = "/Users/jiankaiwang/devops/Auxiliary_Operations/vc_cicd/tmp/new01_retrain_dev/mbv2_0.75_128/retrained_graph.pb"
assert os.path.exists(frozen_model_path), "Frozen model was not found."